<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px; width: 180px">
            <img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo">
        </td>
        <td style="width: 100%">
            <div style="width: 100%; text-align:right"><font style="font-size:38px"><b>Softwaredesign</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>WS 2023</b></font></div>
        </td>
    </tr>
</table>

---

# 01_06_Python Grundlagen - Klassehierarchie zur Datenverarbeitung

Basierend auf dem Gründgerüst der letzen Hausübung, soll nun eine sinnvolle Klassenhierarchie erstellt werden.

Es konnte festgestellt werden, dass die Klasse `MovingAverageProcessor` und `RMSEProcessor` sehr ähnlich sind. Beide Klassen operieren auf `DataContainer`-Objekten, und haben die gleichen Methoden. Abgesehen von der Art der Verarbeitung unterscheiden sie sich in der Anzahl an `DataContainer`-Objekten die sie verarbeiten und in der Tatsache ob ein `DataContainer`-Objekt zurückgegeben wird oder nicht.

Um Redundanz zu vermeiden, soll eine gemeinsame Oberklasse `DataProcessor` erstellt werden, die die gemeinsamen Methoden und Attribute der beiden Klassen enthält. Weiters soll die Klasse `DataProcessorTerminal` erstellt werden die von `DataProcessor` erbt.  
Die Idee dieser Struktur ist es eine eine **Datenpipline** zu definieren. Ein `DataContainer`-Objekt wird durch eine Reihe von `DataProcessor`-Objekten geschickt, die es verarbeiten und das Ergebnis an das nächste `DataProcessor`-Objekt weitergeben. Der letzte `DataProcessor` in der Kette gibt das Ergebnis zurück.  
Die Klasse `DataProcessorTerminal` stellt hierbei üblicherweise den Endpunkt der Kette dar, da sie aus zwei `DataContainer`-Objekten ein Ergebnis berechnet welches nicht weiterverarbeitet werden kann.

## Aufgabenstellung
Es sollen nun die Klassen `DataProcessor`, `DataProcessorTerminal`, `MovingAverageProcessor` und `RMSEProcessor` anhand dieses Klassendiagramms implementiert werden.  
Beachten Sie dabei folgende Punkte:
- `DataProcessor` soll eine abstrakte Klasse sein &rarr; in Pyhton durch erben von `ABC` und nutzen des `@abstractmethod`-Dekorators realisierbar
- `DataProcessorTerminal` soll ebenfalls eine abstrakte Klasse sein
- Integrieren Sie die bestehenden Klassen `MovingAverageProcessor` und `RMSEProcessor` in die neue Klassenhierarchie
- Erstellen Sie die neue Klasse `MAPEProcessor` die den [*Mean Absolute Percentage Error*](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error) berechnet

![](https://mermaid.ink/img/pako:eNqtk1FrgzAQx79KyFO7tV9AiiDrHoWy7mkI5WZOF2YSSc6WzfW7L7W1kE4HQvOUnP-7n_9LruW5Ecgjnlfg3FpCaUFlOtPdma2BYGNNjs4Zy9pMM79Wq-TdkYWc4vgceWT1WTQ7JTwZDSQ12jkLjg-9uETa1eBBSGhncx8_jiBf0SqpoZqMXoToG0Zq9lKXyR4tlHiFtX1FqYkdpBbmsHPyG6d4HLF4Khn8wUu6ff4LLioDxKxyOMncGLQrFxpPNqNYBfX9seEDWi5_4uELHlQOX9ON9PpEupSgr_8qg1bwBVf-K0jhR6HrS8bpAxVmPPJbAfYz496S10FDZvulcx6RbXDBm1oA4WVyeFRA5XwUhSRj08tsGV3I0ufWoN-M6VXHX0GvOYY?type=png)

<!-- 
classDiagram

class DataProcessor {
    <<Abstract>>
    + process(DataConatiner) DataConatiner*
    + get_parameter()*
}

class DataProcessorTerminal {
    <<Abstract>>
    + process(DataConatiner, DataConatiner)*
}

class MovingAverageProcessor{
    + int window_size
    + process(DataConatiner) DataConatiner
    + get_parameter() int
}

class RMSEProcessor{
    + float rmse
    + process(DataConatiner, DataConatiner)
    + get_parameter() float
}

class MAPEProcessor{
    + float mape
    + process(DataConatiner, DataConatiner)
    + get_parameter() float
}

DataProcessor --|> DataProcessorTerminal
DataProcessor --|> MovingAverageProcessor
DataProcessorTerminal --|> RMSEProcessor
DataProcessorTerminal --|> MAPEProcessor
-->

### Notwendige imports

In [35]:
import math
from typing import List, Tuple
from abc import ABC, abstractmethod

import numpy as np
import matplotlib.pyplot as plt

### Definition der `DataContainer` Klasse

Die Klasse ist ident zu jener aus der letzten Hausübung und wird später für das Testen der Funktionalität benötigt

In [36]:
class DataContainer():

    # Konstruktor der Klasse DataContainer
    def __init__(self, x_data: List[float] = [], y_data: List[float] = []) -> None:
        self.x_data = x_data
        self.y_data = y_data

        #Exception werfen wenn Daten nicht gleich lang
        if len(self.x_data) != len(self.y_data):
            raise ValueError("x_data and y_data must have the same length")
    
    def get_x_data(self) -> List[float]:
        return self.x_data
    
    def get_y_data(self) -> List[float]:
        return self.y_data

    def set_x_data(self, x_data: List[float]) -> None:
        self.x_data = x_data

    def set_y_data(self, y_data: List[float]) -> None:
        self.y_data = y_data

    def get_x_mean(self) -> float:
        return sum(self.x_data) / len(self.x_data)

    def get_y_mean(self) -> float:
        return sum(self.y_data) / len(self.y_data)

    # Methode um die Daten zu plotten --> wird in einer späteren Einheit behandelt
    def plot(self) -> None:
        plt.plot(self.x_data, self.y_data, marker="o")
        plt.xlabel('x')
        plt.ylabel('y')

        # Hack to get the name of the instance variable for plotting --> should not be used in practice
        plt.title([k for k,v in globals().items() if v is self][0])        

        plt.show()

### Definition der `DataContainerProcessor` und `DataProcessorTerminal` Klassen

Die Klassen sind abstract und sind nach oben angeführtem Klassendiagramm zu implementieren.

In [37]:
#Lösung hier einfügen:
class DataProcessor(ABC):
    #implementieren Sie die Klasse fertig
    pass

class DataProcessorTerminal(DataProcessor):
    #implementieren Sie die Klasse fertig
    pass

### Klassen zur Datenverarbeitung

Es sollen die Klassen `MovingAverageProcessor` und `RMSEProcessor` im Kontext der Klassenhierarchie implementiert werden.

In [38]:
#Lösung hier einfügen:
class MovingAverageProcessor(DataProcessor):
    #Übertragen Sie Ihre Implementierung in diese vererbte Klasse
    pass

In [39]:
#Lösung hier einfügen:
class RMSEProcessor(DataProcessorTerminal):
    #Übertragen Sie Ihre Implementierung in diese vererbte Klasse
    pass

### Testen der Implementierung

Es ist ein `DataContainer`-Objekt gegeben, welches mit einer Instanz von `MovingAverageProcessor` (mit `window_size = 3`) und `RMSEProcessor` verarbeitet werden soll.  
Als finales Ergebnis ist nur der RMSE-Wert zwischen originalen Daten und den gefilterten Daten von Interesse. Es sollen also keine Zwischenergebnisse gespeichert werden.

In [40]:
x_data = np.linspace(0, 10, 100)
y_data = 2 * x_data + 1 + np.random.normal(0, 1, 100)
almost_lin_data = DataContainer(x_data, y_data)

# Processing pipeline of Moving Average Filter --> RMSE
ma_proc_3 = MovingAverageProcessor(3)
rmse_proc = RMSEProcessor()

#Lösung hier einfügen:


RMSE: 0.9020282498226838


### Implementierungen der neuen Klasse `MAPEProcessor`

Es soll nun die `MAPEProcessor`-Klasse implementiert werden, die den *Mean Absolute Percentage Error* berechnet.

In [41]:
class MAPEProcessor(DataProcessorTerminal):
    #Implementieren Sie die Klasse fertig
    pass

### Testen der Implementierung

Es soll die selbe Datenpipeline wie zuvor erstellt werden, jedoch soll nun der RMSE-Wert zwischen den originalen und den gefilterten Daten durch den MAPE-Wert ersetzt werden.  
Da sowohl der `MAPEProcessor` als auch der `RMSEProcessor` von `DataProcessorTerminal` erben, kann die Datenpipeline einfach umgestellt werden, ohne dass hier Änderungen notwendig sind.

In [42]:
mape_proc = MAPEProcessor()

# Processing pipeline of Moving Average Filter --> RMSE
mape_proc.process(almost_lin_data, ma_proc_3.process(almost_lin_data))
print(f"MAPE: {mape_proc.get_parameter()}")

MAPE: -18.724002983824285


### Sinnvolles Handling der Datenpipeline

Um die Datenpipeline sinnvoll zu handhaben, soll eine Funktion `process_pipeline()` implementiert werden, die eine Liste von `DataProcessor`-Objekten entgegennimmt und weiter verarbeitet. Die Liste an `DataProcessor`-Objekten soll in der Reihenfolge der Liste abgearbeitet werden, wobei das letzte Element vom Typ `DataProcessorTerminal` sein muss.

Die Funktion soll daher die Liste abarbeiten und das Ergebnis der `get_parameter`-Methode des letzten Elements zurückgeben.

In [43]:
def process_pipeline(pipeline: List[DataProcessor], data: DataContainer, data2: DataContainer = None):
    #Implementieren Sie die Funktion fertig
    pass

### Testen Sie die `process_pipeline()`-Funktion

In [45]:
pipline = [ma_proc_3, rmse_proc]
rmse_value = process_pipeline(pipline, almost_lin_data, almost_lin_data)
print(f"RMSE: {rmse_value}")

pipline[-1] = mape_proc
mape_value = process_pipeline(pipline, almost_lin_data, almost_lin_data)
print(f"MAPE: {mape_value}")

RMSE: 0.9020282498226838
MAPE: 9.978958490231816
